# Traffic Violation RAG System
In this exam, you will implement a Retrieval-Augmented Generation (RAG) system that uses a language model and a vector database to answer questions about traffic violations. The goal is to generate answers with relevant data based on a dataset of traffic violations and fines.

Here are helpful resources:
* [LangChain](https://www.langchain.com/)
* [groq cloud documentation](https://console.groq.com/docs/models)
* [LangChain HuggingFace](https://python.langchain.com/docs/integrations/text_embedding/sentence_transformers/)
* [Chroma Vector Store](https://python.langchain.com/docs/integrations/vectorstores/chroma/)
* [Chroma Website](https://docs.trychroma.com/getting-started)
* [ChatGroq LangChain](https://python.langchain.com/docs/integrations/chat/groq/)
* [LLM Chain](https://api.python.langchain.com/en/latest/chains/langchain.chains.llm.LLMChain.html#langchain.chains.llm.LLMChain)

Dataset [source](https://www.moi.gov.sa/wps/portal/Home/sectors/publicsecurity/traffic/contents/!ut/p/z0/04_Sj9CPykssy0xPLMnMz0vMAfIjo8ziDTxNTDwMTYy83V0CTQ0cA71d_T1djI0MXA30gxOL9L30o_ArApqSmVVYGOWoH5Wcn1eSWlGiH1FSlJiWlpmsagBlKCQWqRrkJmbmqRqUZebngB2gUJAKdERJZmqxfkG2ezgAhzhSyw!!/)

Some installs if needed:
```python
!pip install langchain_huggingface langchain langchain-community langchain_chroma Chroma langchain_groq LLMChain
```

In [1]:
!kaggle datasets download -d khaledzsa/dataset
!unzip dataset.zip

Dataset URL: https://www.kaggle.com/datasets/khaledzsa/dataset
License(s): unknown
  0% 0.00/3.73k [00:00<?, ?B/s]
100% 3.73k/3.73k [00:00<00:00, 5.70MB/s]
Archive:  dataset.zip
  inflating: Dataset.csv             


## Step 1: Install Required Libraries

To begin, install the necessary libraries for this project. The libraries include `LangChain` for building language model chains, and `Chroma` for managing a vector database.

In [38]:
%%capture
!pip install langchain
!pip install transformers torch chromadb
!pip install langchain_community
!pip install sentence_transformers
!pip install OpenAI

In [2]:
!pip install langchain_huggingface langchain langchain-community langchain_chroma Chroma langchain_groq LLMChain

  Preparing metadata (setup.py) ... done
ERROR: Could not find a version that satisfies the requirement LLMChain (from versions: none)
ERROR: No matching distribution found for LLMChain


# Step 2: Load the Traffic Violations Dataset

You are provided with a dataset of traffic violations. Load the CSV file into a pandas DataFrame and preview the first few rows of the dataset using `.head()`. You can also try and see the dataset's characteristics.

In [3]:
import pandas as pd
df = pd.read_csv('Dataset.csv')
df.head()

,المخالفة,الغرامة
0,قيادة المركبة في الأسواق التي لا يسمح بالقيادة...,الغرامة المالية 100 - 150 ريال
1,ترك المركبة مفتوحة وفي وضع التشغيل بعد مغادرتها.,الغرامة المالية 100 - 150 ريال
2,عدم وجود تأمين ساري للمركبة.,الغرامة المالية 100 - 150 ريال
3,عبور المشاة للطرق من غير الأماكن المخصصة لهم.,الغرامة المالية 100 - 150 ريال
4,عدم تقيد المشاة بالإشارات الخاصة بهم.,الغرامة المالية 100 - 150 ريال


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104 entries, 0 to 103
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   المخالفة  104 non-null    object
 1   الغرامة   104 non-null    object
dtypes: object(2)
memory usage: 1.8+ KB


## Step 3: Create Markdown Content from the Dataset

For each traffic violation in the dataset, you will generate markdown text that describes the violation and the associated fine. Create a loop to iterate through the dataset and store the generated markdown in a list. Each fine should look like this:

**المخالفة** - الغرامة

In [92]:
markdown_texts = []
for index, row in df.iterrows():
  Column1 = row['المخالفة']
  Column2 = row['الغرامة']
  markdown = f"{Column1} - {Column2}"
  markdown_texts.append(markdown)
markdown_texts[:5]

['قيادة المركبة في الأسواق التي لا يسمح بالقيادة فيها. - الغرامة المالية 100 - 150 ريال',
 'ترك المركبة مفتوحة وفي وضع التشغيل بعد مغادرتها. - الغرامة المالية 100 - 150 ريال',
 'عدم وجود تأمين ساري للمركبة. - الغرامة المالية 100 - 150 ريال',
 'عبور المشاة للطرق من غير الأماكن المخصصة لهم. - الغرامة المالية 100 - 150 ريال',
 'عدم تقيد المشاة بالإشارات الخاصة بهم. - الغرامة المالية 100 - 150 ريال']

In [19]:
markdown_texts

['**قيادة المركبة في الأسواق التي لا يسمح بالقيادة فيها.** - الغرامة المالية 100 - 150 ريال',
 '**ترك المركبة مفتوحة وفي وضع التشغيل بعد مغادرتها.** - الغرامة المالية 100 - 150 ريال',
 '**عدم وجود تأمين ساري للمركبة.** - الغرامة المالية 100 - 150 ريال',
 '**عبور المشاة للطرق من غير الأماكن المخصصة لهم.** - الغرامة المالية 100 - 150 ريال',
 '**عدم تقيد المشاة بالإشارات الخاصة بهم.** - الغرامة المالية 100 - 150 ريال',
 '**وقوف المركبة في أماكن غير مخصصة للوقوف.** - الغرامة المالية 100 - 150 ريال',
 '**عدم إعطاء أفضلية المرور للمشاة أثناء عبورهم في المسارات المخصصة لهم.** - الغرامة المالية 100 - 150 ريال',
 '**عدم استخدام إشارة الالتفاف عند التحول لليمين أو اليسار أو التجاوز أو تغيير المسار.** - الغرامة المالية 150 - 300 ريال',
 '**الرجوع إلى الخلف في الطريق العام لمسافة تزيد على عشرين متراً.** - الغرامة المالية 150 - 300 ريال',
 '**قيام سائق الدراجة الآلية أو العادية - أو ما في حكمهما - بالتعلق بأي مركبة أخرى، أو سحب أو حمل أشياء تعرض مستخدمي الطريق للخطر.** - الغرامة المالية 150 - 300 ر

## Step 4: Chunk the Markdown Data

Using LangChain's `RecursiveCharacterTextSplitter`, split the markdown texts into smaller chunks that will be stored in the vector database.

In [21]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(chunk_size=200, add_start_index=True)
chunks = splitter.create_documents(markdown_texts)

In [26]:
chunks[10]

Document(metadata={'start_index': 0}, page_content='**مخالفة قواعد استعمال أنوار التلاقي.** - الغرامة المالية 150 - 300 ريال')

In [93]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, length_function=len,
    is_separator_regex=False, add_start_index=True
)

In [96]:
markdown_texts[103]

'المجازفة بعبور الأودية والشعاب أثناء جريانها. - الغرامة المالية 5000 - 10000 ريال'

## Step 5: Generate Embeddings for the Documents

Generate embeddings for the chunks of text using HuggingFace's pre-trained Arabic language model. These embeddings will be stored in a `Chroma` vector store.

In [28]:
from transformers import AutoModelForMaskedLM
model = AutoModelForMaskedLM.from_pretrained("asafaya/bert-base-arabic")
from transformers import AutoTokenizer, AutoModel
import torch

tokenizer = AutoTokenizer.from_pretrained("asafaya/bert-base-arabic")
model = AutoModel.from_pretrained("asafaya/bert-base-arabic")

def generate_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
        embeddings = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()
    return embeddings
embeddings = [generate_embedding(chunk.page_content) for chunk in chunks]

Some weights of the model checkpoint at asafaya/bert-base-arabic were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/62.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/334k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [97]:
# https://python.langchain.com/docs/integrations/vectorstores/chroma/
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass()

from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

··········


# Step 6: Define the RAG Prompt Template

Define a custom prompt template in Arabic to retrieve traffic violation-related answers based on the context. Ensure the template encourages the model to give **advice** in **Arabic**, staying within the context provided.

In [75]:
from langchain.prompts import PromptTemplate
prompt_template = """
أنت مساعد ذكي. وظيفتك تقديم نصائح مرورية استناداً إلى البيانات المتاحة حول المخالفات المرورية والغرامات المرتبطة بها. رجاءً أجب عن السؤال التالي استناداً إلى السياق المتوفر:

السؤال: {query}

السياق: {context}

الإجابة:
"""
rag_prompt_template = PromptTemplate(input_variables=["query", "context"], template=prompt_template)

## Step 7: Initialize the Language Model

Initialize the language model using the Groq API. Set up the model with a specific configuration, including the API key, temperature setting, and model name.

In [76]:
import getpass
import os

if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API key: ")

In [77]:
%pip install -qU langchain-groq

In [78]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="mixtral-8x7b-32768",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)
print(llm)

client=<groq.resources.chat.completions.Completions object at 0x7901f6fc69e0> async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7901f6fc5b40> temperature=1e-08 model_kwargs={} groq_api_key=SecretStr('**********')


## Step 8: Create the LLM Chain


Now, you will create an LLM Chain that combines the language model and the prompt template you defined. This chain will be used to generate responses based on the retrieved context.

In [79]:
from langchain.chains import LLMChain
llm_chain = LLMChain(
    llm=llm,
    prompt=rag_prompt_template
)

response = llm_chain.run({
    "query": "ما هي الغرامة على القيادة بدون رخصة؟",
    "context": "النص الخاص بالغرامات المرورية "
})
print(response)

استناداً إلى السياق المتاح، لا يتم تقديم إجابة مباشرة للسؤال. إلا أنني قد بحثت في مواقع عامة ووثائق قانونية وأجد أن الغرامة على القيادة بدون رخصة يمكن أن تتراوح بين 100 و 1000 دولارات أمريكية (USD) حسب الدولة والحالة. يمكنك زيارة موقع الحكومة المحلية للحصول على المعلومات المحدثة والدقيقة حول هذه المحكمة.


## Step 9: Implement the Query Function

Create a function `query_rag` that will take a user query as input, retrieve relevant context from the vector store, and use the language model to generate a response based on that context.

In [80]:
def query_rag(user_query, Column1, llm_chain):
    docs = Column1.similarity_search(user_query, k=5)
    context = "\n".join([doc.page_content for doc in docs])
    response = llm_chain.run({
        "query": user_query,
        "context": context
    })

    return response

## Step 10: Inference - Running Queries in the RAG System

In this final step, you will implement an inference pipeline to handle real-time queries. You will allow the system to retrieve the most relevant violations and fines based on a user's input and generate a response.

1. Inference Workflow:

  * The user inputs a query (e.g., "ماهي الغرامة على القيادة بدون رخصة؟").
  * The system searches for the most relevant context from the traffic violation vector store.
  * It generates an answer and advice based on the context.

2. Goal:
  * Run the inference to answer questions based on the traffic violation dataset.

In [81]:
def inference_pipeline(user_query):
    # The Step 1
    response = query_rag(user_query, Column1, llm_chain)

    # The Step 2
    print(f"Response: {response}")

user_query = "ما هي الغرامة على القيادة بدون رخصة؟"
inference_pipeline(user_query)


AttributeError: 'str' object has no attribute 'similarity_search'